In [3]:
import math
from collections import defaultdict
import networkx as nx
from svgpathtools import svg2paths2

def sample_path(path, step=2.0):
    """Sample complex path into (x,y) points spaced by ~step."""
    L = path.length()
    if L == 0:
        return []
    n = max(2, int(math.ceil(L / step)))
    ts = [i / (n - 1) for i in range(n)]
    pts = []
    for t in ts:
        z = path.point(t)
        pts.append((float(z.real), float(z.imag)))
    return pts

def snap_point(p, snap=1.5):
    """Snap (x,y) to a grid of size `snap` to merge near-duplicates."""
    x, y = p
    return (round(x / snap) * snap, round(y / snap) * snap)

def svg_to_graph(svg_file, step=2.0, snap=1.5, min_segment=0.0):
    """
    Convert SVG paths to an undirected graph.
    - step: sampling step along curves (in SVG units)
    - snap: snapping tolerance (grid size) to merge close points
    - min_segment: discard segments shorter than this length
    """
    paths, attrs, svg_attr = svg2paths2(svg_file)

    # 1) Collect polylines from all paths
    polylines = []
    for path in paths:
        pts = sample_path(path, step=step)
        if len(pts) >= 2:
            polylines.append(pts)

    # 2) Snap points and split into segments
    G = nx.Graph()
    id_map = {}  # snapped point -> node id
    def get_node_id(pt):
        if pt not in id_map:
            node_id = len(id_map)
            id_map[pt] = node_id
            G.add_node(node_id, pos=pt)
        return id_map[pt]

    def seg_len(a, b):
        return math.hypot(a[0] - b[0], a[1] - b[1])

    for poly in polylines:
        snapped = [snap_point(p, snap) for p in poly]
        # de-duplicate consecutive identical snapped points
        clean = [snapped[0]]
        for p in snapped[1:]:
            if p != clean[-1]:
                clean.append(p)
        if len(clean) < 2:
            continue

        # add edges between consecutive points
        for i in range(len(clean) - 1):
            a, b = clean[i], clean[i + 1]
            if seg_len(a, b) <= min_segment:
                continue
            u, v = get_node_id(a), get_node_id(b)
            # Store the small segment; we can later collapse degree-2 chains if desired
            if G.has_edge(u, v):
                # Optionally append geometry to a list; here we keep last
                G[u][v]['polyline'] = [a, b]
            else:
                G.add_edge(u, v, polyline=[a, b])

    # 3) Optional: collapse degree-2 chains to simplify graph geometry
    #    (keeps only junctions and endpoints as nodes)
    def collapse_chains(G):
        H = G.copy()
        changed = True
        while changed:
            changed = False
            for n in list(H.nodes()):
                if H.degree(n) == 2 and H.number_of_nodes() > 2:
                    nbrs = list(H.neighbors(n))
                    if len(nbrs) != 2:
                        continue
                    a, b = nbrs
                    if a == b or H.has_edge(a, b):
                        continue
                    # merge polylines (a-n) + (n-b)
                    pa = H[a][n].get('polyline', [H.nodes[a]['pos'], H.nodes[n]['pos']])
                    pb = H[n][b].get('polyline', [H.nodes[n]['pos'], H.nodes[b]['pos']])
                    # ensure continuity: remove duplicate middle point
                    merged = pa + pb[1:]
                    H.add_edge(a, b, polyline=merged)
                    H.remove_node(n)
                    changed = True
                    break
        return H

    G_simplified = collapse_chains(G)
    return G_simplified


In [29]:
# Example usage:
G = svg_to_graph("/rdf/rise/Houbo/Code_EMX_SiGe/SVG_TEMP/XFMR8Shaped1x20.svg", step=50, snap=12, min_segment=8)
pos = nx.get_node_attributes(G, "pos")  # dict: node -> (x, y)
for u, v, data in G.edges(data=True):
    poly = data["polyline"]  # list of (x, y) points for that edg

In [30]:
print(poly)

[(-2280, -3900), (-2280, -3948)]


In [31]:
print(pos)

{175: (-1884, -924), 181: (-1584, -924), 465: (-1488, -936), 466: (-1524, -924), 471: (-1824, -924), 681: (2004, 3624), 682: (2004, 3672), 711: (-744, 1200), 712: (-696, 1200), 976: (-84, 936), 1067: (756, 996), 1069: (756, 1080), 1127: (228, 960), 1131: (-768, 996), 1132: (-720, 996), 1133: (-696, 1032), 1187: (768, 876), 1188: (720, 876), 1190: (696, 804), 1275: (696, 600), 1276: (744, 600), 1277: (768, 624), 1407: (696, 996), 1413: (1008, 1116), 1785: (-144, 3360), 1786: (-144, 3408), 1852: (768, 828), 1856: (696, 1200), 1857: (744, 1200), 1858: (768, 1224), 1862: (-768, 804), 1863: (-720, 804), 1864: (-696, 828), 2288: (-2280, -3852), 2289: (-2280, -3900), 2290: (-2280, -3948)}


In [28]:
len(pos)

36

In [33]:
for u, v, data in G.edges(data=True):
    print(u, v)

175 471
175 682
175 181
175 681
181 466
181 465
181 471
465 466
681 682
711 712
711 976
711 1188
712 976
712 1190
976 1786
976 1785
1067 1407
1067 1127
1067 1069
1067 1413
1069 1127
1069 1413
1069 1407
1131 1132
1131 1133
1132 1133
1187 1188
1187 1852
1187 1190
1188 1190
1190 1852
1275 1276
1275 1277
1276 1277
1785 1786
1856 1857
1856 1858
1857 1858
1862 1863
1862 1864
1863 1864
2288 2289
2288 2290
2289 2290
